# Agrupación y limpieza de partidas

In [1]:
#!pip install python-chess
#!pip install tqdm

     |████████████████████████████████| 78 kB 1.4 MB/s eta 0:00:01


In [1]:
# Paquetes necesarios
import chess.pgn
import os
import shutil
from collections import defaultdict
from datetime import datetime
#from tqdm import tqdm
import subprocess
from pyspark.sql import SparkSession
from io import StringIO
import uuid
from multiprocessing import Pool
import shutil
from collections import defaultdict
import subprocess


## 1. Agrupación de partidas por trimestres

Esto simplificará el número de ficheros y lo hace más apto para su almacén en HDFS (bloques de 128 Mb)

In [2]:
!hdfs dfs -rm -r /user/ajedrez/semestres

Deleted /user/ajedrez/semestres


In [3]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("AgruparPartidasLocal")
    .master("local[*]")    ## "yarn" es mucho más lento y no necesario (ficheros .pgn muy pequeños)
    .getOrCreate()
)

sc = spark.sparkContext

print("Sesión iniciada en modo:", sc.master)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/09 21:37:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✅ Sesión iniciada en modo: local[*]


In [4]:
# Función para procesar contenido PGN
def procesar_contenido_pgn(contenido):
    partidas_por_semestre = defaultdict(list)
    f = StringIO(contenido)

    while True:
        try:
            game = chess.pgn.read_game(f)
            if game is None:
                break

            fecha_str = game.headers.get("Date", "")
            try:
                fecha = datetime.strptime(fecha_str, "%Y.%m.%d")
                anio = fecha.year
                mes = fecha.month
                semestre = (mes - 1) // 6 + 1
            except:
                continue

            exporter = chess.pgn.StringExporter(headers=True, variations=True, comments=True)
            partida_str = game.accept(exporter)
            partidas_por_semestre[(anio, semestre)].append(partida_str)
        except:
            continue

    resultados = []
    for (anio, semestre), partidas in partidas_por_semestre.items():
        for partida in partidas:
            resultados.append(((anio, semestre), partida))
    return resultados

# Leer archivos PGN desde HDFS
rdd = sc.wholeTextFiles("hdfs:///user/ajedrez/raw/*.pgn")

# Procesar y agrupar por semestre
rdd_partidas = rdd.flatMap(lambda x: procesar_contenido_pgn(x[1]))
rdd_agrupado = rdd_partidas.groupByKey().mapValues(list)

# Guardar cada grupo como archivo en HDFS
def guardar_en_archivo_hdfs(clave_partidas):
    (anio, semestre), partidas = clave_partidas
    nombre_archivo = f"/user/ajedrez/semestres/partidas_{anio}_S{semestre}.pgn"
    contenido = "\n\n".join(partidas)

    tmp_path = f"/tmp/{anio}_S{semestre}.pgn"
    with open(tmp_path, "w", encoding="utf-8") as f:
        f.write(contenido)

    os.system(f"hdfs dfs -mkdir -p /user/ajedrez/semestres")
    os.system(f"hdfs dfs -put -f {tmp_path} {nombre_archivo}")
    os.remove(tmp_path)

# Ejecutar la escritura
rdd_agrupado.foreach(guardar_en_archivo_hdfs)

In [5]:
!hdfs dfs -ls /user/ajedrez/semestres

Found 3 items
-rw-r--r--   2 root supergroup    5936129 2025-05-09 21:41 /user/ajedrez/semestres/partidas_2024_S1.pgn
-rw-r--r--   2 root supergroup   15201222 2025-05-09 21:41 /user/ajedrez/semestres/partidas_2024_S2.pgn
-rw-r--r--   2 root supergroup    5757426 2025-05-09 21:41 /user/ajedrez/semestres/partidas_2025_S1.pgn


## 2. Quitar partidas de 960 (Freechess style)

In [ ]:
'''# Crear sesión de Spark
spark = SparkSession.builder \
    .appName("FiltrarPGNs") \
    .master("local[*]") \
    .getOrCreate()

sc = spark.sparkContext

spark.stop()'''

In [1]:
!hdfs dfs -rm -r /user/ajedrez/filtrados

rm: `/user/ajedrez/filtrados': No such file or directory


In [1]:
from pyspark.sql import SparkSession
from datetime import datetime
from io import StringIO
import chess.pgn
import os

# 🔹 Crear sesión Spark
spark = (
    SparkSession.builder
    .appName("FiltrarPartidasNo960")
    .master("yarn")  # Ejecuta los jobs en el cluster Spark-YARN
    .getOrCreate()
)
# 🔹 Rutas
ruta_entrada = "hdfs:///user/ajedrez/semestres/*.pgn"
ruta_salida = "/user/ajedrez/filtrados/"

# 🔹 Leer archivos (ruta, contenido)
archivos_pgn = sc.wholeTextFiles(ruta_entrada)

# 🔹 Filtrar partidas válidas
def filtrar_partidas(nombre_y_contenido):
    ruta_archivo, contenido = nombre_y_contenido
    nombre_base = os.path.basename(ruta_archivo)
    salida = []

    for bloque in contenido.split("\n\n"):
        bloque = bloque.strip()
        if not bloque:
            continue
        try:
            game = chess.pgn.read_game(StringIO(bloque))
            if game is None:
                continue

            # Validar movimientos
            board = game.board()
            for move in game.mainline_moves():
                board.push(move)

            variante = game.headers.get("Variant", "").lower()
            if variante not in ["chess960", "960"]:
                exporter = chess.pgn.StringExporter(headers=True, variations=True, comments=True)
                partida_str = game.accept(exporter)
                salida.append((nombre_base, partida_str))
        except Exception:
            continue  # Ignorar errores

    return salida

# 🔹 Aplicar filtrado
filtrados = archivos_pgn.flatMap(filtrar_partidas)

# 🔹 Agrupar y filtrar archivos vacíos
agrupados = (
    filtrados
    .groupByKey()
    .mapValues(lambda partidas: "\n\n".join(partidas))
    .filter(lambda x: x[1].strip() != "")
)

# 🔹 Crear carpeta de salida si no existe
os.system(f"hdfs dfs -mkdir -p {ruta_salida}")

# 🔹 Guardar en HDFS
def guardar_archivo(nombre_contenido):
    nombre, contenido = nombre_contenido
    nombre_salida = f"filtrado_{nombre}"
    ruta_hdfs = os.path.join(ruta_salida, nombre_salida)
    ruta_tmp = f"/tmp/{nombre_salida}"

    with open(ruta_tmp, "w", encoding="utf-8") as f:
        f.write(contenido)

    os.system(f"hdfs dfs -rm -f {ruta_hdfs}")
    os.system(f"hdfs dfs -put -f {ruta_tmp} {ruta_hdfs}")
    os.remove(ruta_tmp)

agrupados.foreach(guardar_archivo)

print("✅ Archivos filtrados guardados en una sola carpeta HDFS.")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/09 22:22:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
illegal san: 'Nb6' in rnbqkbnr/pppppppp/8/8/5P2/8/PPPPP1PP/RNBQKBNR b KQkq - 0 1 while parsing <Game at 0x7a16ba00f880 ('?' vs. '?', '????.??.??' at '?')>
illegal san: 'Nb3' in rnbqkb1r/pp2pp2/2p2np1/2Pp3p/3P3P/6P1/PP2PP2/RNBQKBNR w KQkq - 0 6 while parsing <Game at 0x7a16ba3017f0 ('?' vs. '?', '????.??.??' at '?')>
[Stage 1:=============================>                             (1 + 1) / 2]

✅ Archivos filtrados guardados en una sola carpeta HDFS.


In [5]:
!hdfs dfs -ls /user/ajedrez/filtrados

Found 3 items
-rw-r--r--   2 root supergroup    6612589 2025-05-09 22:25 /user/ajedrez/filtrados/filtrado_partidas_2024_S1.pgn
-rw-r--r--   2 root supergroup   16935223 2025-05-09 22:25 /user/ajedrez/filtrados/filtrado_partidas_2024_S2.pgn
-rw-r--r--   2 root supergroup    6411718 2025-05-09 22:25 /user/ajedrez/filtrados/filtrado_partidas_2025_S1.pgn


In [8]:
!hdfs dfs -get /user/ajedrez/filtrados/*.pgn /shared

#Archivos con errores

In [9]:
#rm /shared/*.pgn

In [10]:
!hdfs dfs -ls /user/ajedrez/filtrados

Found 3 items
-rw-r--r--   2 root supergroup    6612591 2025-05-09 21:45 /user/ajedrez/filtrados/filtrado_partidas_2024_S1.pgn
-rw-r--r--   2 root supergroup   16935225 2025-05-09 21:45 /user/ajedrez/filtrados/filtrado_partidas_2024_S2.pgn
-rw-r--r--   2 root supergroup    6411720 2025-05-09 21:45 /user/ajedrez/filtrados/filtrado_partidas_2025_S1.pgn


## 3. Eliminación de partidas con errores

En algunos archivos .pgn, se pueden encontrar partidas incompletas o corruptas, jugadas ilegales (por ejemplo, por errores de edición), encabezados mal formados, etc. Limpiaremos estas partidas con errores usando `python-chess`, ya que al intentar reproducir las jugadas en un `chess.Board()`, lanza excepciones si hay jugadas inválidas.

In [5]:
from pyspark.sql import SparkSession
from io import StringIO
import os
import chess.pgn
import subprocess

# Inicializar sesión Spark
spark = SparkSession.builder \
    .appName("LimpiarPartidasNoValidas") \
    .getOrCreate()
sc = spark.sparkContext

# Rutas en HDFS
ruta_entrada = "/user/ajedrez/filtrados/"       
ruta_salida = "/user/ajedrez/limpios/"

# Asegurar carpeta de salida
subprocess.run(["hdfs", "dfs", "-mkdir", "-p", ruta_salida])

# Leer archivos .pgn completos desde HDFS
archivos_pgn = sc.wholeTextFiles(ruta_entrada + "*.pgn")  # (ruta_archivo, contenido)

def limpiar_partidas_validas(ruta_y_contenido):
    ruta_archivo, contenido = ruta_y_contenido
    nombre_archivo = os.path.basename(ruta_archivo)
    salida = []

    pgn_stream = StringIO(contenido)
    while True:
        try:
            game = chess.pgn.read_game(pgn_stream)
            if game is None:
                break

            board = game.board()
            try:
                for move in game.mainline_moves():
                    board.push(move)
            except Exception:
                continue  # Partida inválida

            # Partida válida → exportarla
            exporter = chess.pgn.StringExporter(headers=True, variations=True, comments=True)
            salida.append((nombre_archivo, game.accept(exporter) + "\n\n"))

        except Exception:
            continue  # Error crítico

    return salida

# Filtrar y agrupar por archivo
partidas_limpias = archivos_pgn.flatMap(limpiar_partidas_validas)
por_archivo = partidas_limpias.groupByKey().mapValues(lambda partidas: "".join(partidas))

# Guardar resultados
for nombre, contenido in por_archivo.collect():
    print(f"Guardando limpio_{nombre}...")
    nombre_limpio = f"limpio_{nombre}"
    ruta_local_temp = f"/tmp/{nombre_limpio}"
    ruta_hdfs = f"{ruta_salida}{nombre_limpio}"

    with open(ruta_local_temp, "w", encoding="utf-8") as f:
        f.write(contenido)

    subprocess.run(["hdfs", "dfs", "-rm", ruta_hdfs], stderr=subprocess.DEVNULL)
    subprocess.run(["hdfs", "dfs", "-put", ruta_local_temp, ruta_hdfs])
    print(f"Subido: {ruta_hdfs}")

spark.stop()


Guardando limpio_filtrado_partidas_2022_3.pgn...
Subido: /user/ajedrez/limpios/limpio_filtrado_partidas_2022_3.pgn
Guardando limpio_filtrado_partidas_2024_3.pgn...
Subido: /user/ajedrez/limpios/limpio_filtrado_partidas_2024_3.pgn
Guardando limpio_filtrado_partidas_2023_1.pgn...
Subido: /user/ajedrez/limpios/limpio_filtrado_partidas_2023_1.pgn
Guardando limpio_filtrado_partidas_2024_1.pgn...
Subido: /user/ajedrez/limpios/limpio_filtrado_partidas_2024_1.pgn
Guardando limpio_filtrado_partidas_2025_1.pgn...
Subido: /user/ajedrez/limpios/limpio_filtrado_partidas_2025_1.pgn


### Sobrante: Funciona pero usa collect()

In [2]:
## FUNCIONA
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("AgruparPartidasLocal")
    .master("local[*]")    ## "yarn" es mucho más lento y no necesario (ficheros .pgn muy pequeños)
    .getOrCreate()
)

sc = spark.sparkContext

print("✅ Sesión iniciada en modo:", sc.master)


# Crear carpeta local de salida
output_dir = "./partidas_agrupadas"
os.makedirs(output_dir, exist_ok=True)

# Leer rutas únicas de archivos PGN en HDFS
rdd = sc.wholeTextFiles("hdfs:///user/ajedrez/raw/*.pgn")
archivos = rdd.map(lambda x: x[0]).distinct().collect()

# Acumulador de resumen
resumen_cuatrimestres = defaultdict(int)

def procesar_contenido_pgn(contenido):
    partidas_por_cuatrimestre = defaultdict(list)
    f = StringIO(contenido)

    while True:
        try:
            game = chess.pgn.read_game(f)
            if game is None:
                break

            fecha_str = game.headers.get("Date", "")
            try:
                fecha = datetime.strptime(fecha_str, "%Y.%m.%d")
                anio = fecha.year
                mes = fecha.month
                cuatrimestre = (mes - 1) // 4 + 1
            except:
                continue

            exporter = chess.pgn.StringExporter(headers=True, variations=True, comments=True)
            partida_str = game.accept(exporter)
            partidas_por_cuatrimestre[(anio, cuatrimestre)].append(partida_str)
        except:
            continue

    return partidas_por_cuatrimestre

# Procesar cada archivo PGN
for ruta in archivos:
    print(f"📂 Procesando archivo: {ruta}")
    contenido = rdd.filter(lambda x: x[0] == ruta).collect()[0][1]
    agrupadas = procesar_contenido_pgn(contenido)

    for (anio, cuatrimestre), partidas in agrupadas.items():
        nombre_archivo = f"partidas_{anio}_{cuatrimestre}.pgn"
        ruta_local = os.path.join(output_dir, nombre_archivo)

        with open(ruta_local, "a", encoding="utf-8") as f_out:
            for partida in partidas:
                f_out.write(partida + "\n\n")

        resumen_cuatrimestres[(anio, cuatrimestre)] += len(partidas)
        print(f"✅ Añadidas {len(partidas)} partidas a {ruta_local}")

# Subir todos los archivos locales generados a HDFS
hdfs_destino = "/user/ajedrez/cuatrimestres"
os.system(f"hdfs dfs -mkdir -p {hdfs_destino}")

for archivo in os.listdir(output_dir):
    if archivo.endswith(".pgn"):
        ruta_local = os.path.join(output_dir, archivo)
        ruta_hdfs = f"{hdfs_destino}/{archivo}"
        os.system(f"hdfs dfs -put -f {ruta_local} {ruta_hdfs}")
        print(f"☁️ Subido: {ruta_local} → {ruta_hdfs}")

# Mostrar resumen final
print("\n📊 Resumen de partidas por cuatrimestre:")
for (anio, cuatrimestre), total in sorted(resumen_cuatrimestres.items()):
    print(f"🗂️  {anio} - Cuatrimestre {cuatrimestre}: {total} partidas")


📂 Procesando archivo: hdfs://namenode:9000/user/ajedrez/raw/twic1469.pgn
✅ Añadidas 10051 partidas a ./partidas_agrupadas/partidas_2022_3.pgn
✅ Añadidas 576 partidas a ./partidas_agrupadas/partidas_2023_1.pgn
📂 Procesando archivo: hdfs://namenode:9000/user/ajedrez/raw/twic1535.pgn
✅ Añadidas 6640 partidas a ./partidas_agrupadas/partidas_2024_1.pgn
📂 Procesando archivo: hdfs://namenode:9000/user/ajedrez/raw/twic1569.pgn
✅ Añadidas 9947 partidas a ./partidas_agrupadas/partidas_2024_3.pgn
✅ Añadidas 35 partidas a ./partidas_agrupadas/partidas_2024_1.pgn
📂 Procesando archivo: hdfs://namenode:9000/user/ajedrez/raw/twic1589.pgn
✅ Añadidas 10235 partidas a ./partidas_agrupadas/partidas_2025_1.pgn
📂 Procesando archivo: hdfs://namenode:9000/user/ajedrez/raw/twic1590.pgn
✅ Añadidas 6431 partidas a ./partidas_agrupadas/partidas_2025_1.pgn
📂 Procesando archivo: hdfs://namenode:9000/user/ajedrez/raw/twic1534.pgn
✅ Añadidas 7971 partidas a ./partidas_agrupadas/partidas_2024_1.pgn
☁️ Subido: ./partid